# 7.1 处理缺失值

In [4]:
import pandas as pd
import numpy as np

In [5]:
string_data = pd.Series(['afdsa', 'fdsa', np.nan, 'fdsa'])
string_data

0    afdsa
1     fdsa
2      NaN
3     fdsa
dtype: object

In [6]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [8]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 过滤缺失值 
用dropna方法

In [12]:
from numpy import nan as NA

##### Series情况

In [10]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [11]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

##### DataFrame情况

In [13]:
data = pd.DataFrame([[1, 2, 3], [1, NA, NA], [NA, NA, NA], [NA, 2, 3]])
data

,0,1,2
0,1.0,2.0,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,2.0,3.0


In [15]:
# 默认删除含有NA的行
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,2.0,3.0


In [16]:
# 只删除全部NA的行
data.dropna(how='all')

,0,1,2
0,1.0,2.0,3.0
1,1.0,NaN,NaN
3,NaN,2.0,3.0


In [18]:
# 删除含有NA的列
data.dropna(axis=1)

""
0
1
2
3


In [19]:
# 保留一定数量的观察值的行
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.636376,NaN,NaN
1,-0.157108,NaN,NaN
2,0.998880,NaN,-0.864128
3,-0.815961,NaN,-0.588463
4,-0.015605,-0.132325,0.290671
5,0.436269,0.484419,0.705654
6,-1.100809,0.239879,-0.733710


In [20]:
df.dropna(thresh=2)

,0,1,2
2,0.998880,NaN,-0.864128
3,-0.815961,NaN,-0.588463
4,-0.015605,-0.132325,0.290671
5,0.436269,0.484419,0.705654
6,-1.100809,0.239879,-0.733710


### 7.1.2 补全缺失值 
用fillna方法

In [22]:
df.fillna(0)

,0,1,2
0,0.636376,0.000000,0.000000
1,-0.157108,0.000000,0.000000
2,0.998880,0.000000,-0.864128
3,-0.815961,0.000000,-0.588463
4,-0.015605,-0.132325,0.290671
5,0.436269,0.484419,0.705654
6,-1.100809,0.239879,-0.733710


In [23]:
# 字典方式为不同列指定不同填充值
df.fillna({1:0.5, 2:0})

,0,1,2
0,0.636376,0.500000,0.000000
1,-0.157108,0.500000,0.000000
2,0.998880,0.500000,-0.864128
3,-0.815961,0.500000,-0.588463
4,-0.015605,-0.132325,0.290671
5,0.436269,0.484419,0.705654
6,-1.100809,0.239879,-0.733710


In [26]:
# fillna返回的是一个新的对象，可以修改原df
df.fillna(0, inplace=True)
df

,0,1,2
0,0.636376,0.000000,0.000000
1,-0.157108,0.000000,0.000000
2,0.998880,0.000000,-0.864128
3,-0.815961,0.000000,-0.588463
4,-0.015605,-0.132325,0.290671
5,0.436269,0.484419,0.705654
6,-1.100809,0.239879,-0.733710


In [27]:
# Series可以填充中位数、平均值等。

# 7.2 数据转换 

###  7.2.1 删除重复值

In [28]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [31]:
# 查看重复行
# duplicated() 返回的是一个布尔Series
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [36]:
# drop_duplicates() 返回的是一个DataFrame
data.drop_duplicates()

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [33]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [34]:
data.drop_duplicates('k1')

,k1,k2,v1
0,one,1,0
1,two,1,1


In [37]:
# drop_duplicates()和 duplicated()默认保留第一个观测到的值，keep='last'保留最后一个

###  7.2.2 使用函数或映射进行数据转换

In [40]:
data = pd.DataFrame({
    'food': ['bacon', 'pulled pork', 'Bacon', 'honey ham'],
    'ounces': [4, 3, 2, 1]
})
data

,food,ounces
0,bacon,4
1,pulled pork,3
2,Bacon,2
3,honey ham,1


In [39]:
meat_to_animal = {
    'bacon': 'pig', 
    'pulled pork': 'pig',
    'honey ham': 'pig'
}

In [44]:
# 先将大写转小写，再用Series的map函数
lowercased = data['food'].str.lower()
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4,pig
1,pulled pork,3,pig
2,Bacon,2,pig
3,honey ham,1,pig


### 7.2.3 替代值 

In [45]:
# 用replace方法替换，会生成新的Series，用inplace=True修改原有Series
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [47]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [48]:
data.replace([-999, 1], np.nan)

0       NaN
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [50]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [51]:
# 也可以传字典参数
data.replace({-999: np.nan})

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

### 7.2.4 重命名轴索引

In [54]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [61]:
# 将索引重命名大写
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [62]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [63]:
# rename 可以不修改原数据集
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [64]:
# 也可以传入字典参数
data.rename(index={'OHIO': 'HELLO'})

,one,two,three,four
HELLO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 离散化和分箱
连续值经常需要离散化，或者分离成“箱子”进行分析。

In [67]:
# 将各年龄段分组
ages = [20, 23,55,43,21,55,66,77,33,99,20,30,50]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats
# 返回的是一个Categorical对象，输出描述了每个数对应的箱

[(18, 25], (18, 25], (35, 60], (35, 60], (18, 25], ..., (25, 35], (60, 100], (18, 25], (25, 35], (35, 60]]
Length: 13
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [71]:
# 对应箱的序号
cats.codes

array([0, 0, 2, 2, 0, 2, 3, 3, 1, 3, 0, 1, 2], dtype=int8)

In [73]:
# 表示箱名的字符串数组
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [74]:
# 对箱的数量计数
pd.value_counts(cats)

(35, 60]     4
(18, 25]     4
(60, 100]    3
(25, 35]     2
dtype: int64

In [75]:
# right=False 改变哪一边封闭
# labels传入列表或数据自定义箱名
# 传入整数箱边，则根据max和min计算出等长的箱
# qcut可以使分箱后，每个箱内的数量相同

### 7.2.6 检测和过滤异常值

In [83]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.041696,0.059426,-0.025370,-0.006779
std,1.012318,1.022631,1.013718,1.010379
min,-3.390039,-3.244345,-4.079423,-4.000144
25%,-0.610633,-0.588491,-0.676641,-0.657805
50%,0.028390,0.047397,-0.039689,-0.023327
75%,0.704306,0.684940,0.605902,0.672077
max,3.557085,3.612510,3.364692,3.439841


In [88]:
col = data[2]
col[np.abs(col)>3]

62    -3.020600
594   -3.274158
797    3.364692
819   -3.278671
896    3.180156
948   -4.079423
Name: 2, dtype: float64

In [93]:
# any方法会过滤掉不符合条件的行，不加any其他行会显示NaN
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
13,-3.239221,1.361863,0.537394,-1.054982
62,1.037123,-0.864440,-3.020600,-0.636579
161,0.090934,3.012496,0.499156,0.800672
180,0.961391,-0.586016,-0.619398,3.439841
223,3.458920,-0.034698,0.690171,-2.093869
313,0.470930,-3.244345,-0.570083,-0.037581
378,-0.684124,-1.887801,-0.532620,-4.000144
488,-3.390039,-0.909099,0.414299,0.212752
538,3.557085,0.627124,0.050466,1.573701
594,1.176487,0.730507,-3.274158,-0.440355


In [101]:
# sign方法会根据值的正负生成1和-1的数值
data[np.abs(data) > 3] = np.sign(data) * 3
data[(np.abs(data) == 3).any(1)]

,0,1,2,3
13,-3.000000,1.361863,0.537394,-1.054982
62,1.037123,-0.864440,-3.000000,-0.636579
161,0.090934,3.000000,0.499156,0.800672
180,0.961391,-0.586016,-0.619398,3.000000
223,3.000000,-0.034698,0.690171,-2.093869
313,0.470930,-3.000000,-0.570083,-0.037581
378,-0.684124,-1.887801,-0.532620,-3.000000
488,-3.000000,-0.909099,0.414299,0.212752
538,3.000000,0.627124,0.050466,1.573701
594,1.176487,0.730507,-3.000000,-0.440355


### 7.2.7 置换和随机抽样

In [102]:
# 使用numpy.random.permutation对DataFrame中的Series或行进行置换。
# Numpy.random中shuffle与permutation的区别：shuffle在原数组操作，permutation返回一个打乱顺序的新数组
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 3, 2, 4])

In [103]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [104]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
4,16,17,18,19


In [106]:
# 随机选取若干行
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19


In [109]:
# 生成一个带有替代值的样本，允许重复选择replace=True
choices = pd.Series([5,2,3,-2,4])
draws = choices.sample(n=10, replace=True)
draws

1    2
4    4
3   -2
1    2
2    3
4    4
1    2
0    5
3   -2
2    3
dtype: int64

### 7.2.8 计算指标/虚拟变量

In [111]:
# pd.get_dummies，将DataFrame中一列有k个不同的值，可以衍生一个k列的值为1和0的矩阵或DataFrame.
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [112]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [115]:
# 加上前缀
pd.get_dummies(df['key'], prefix='key')

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [116]:
# 使用全0的DataFrame是构建指标DataFrame的一种方式。

In [117]:
# 将get_dummies和cut等离散化函数结合使用是统计应用的一种有用方法.
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [119]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

# 7.3 字符串操作

### 7.3.1 字符串对象方法

In [123]:
# split方法根据分隔符拆分多块
# strip方法清除空格（包括换行）
val = 'a,b,  guido'
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [124]:
# join方法连接子字符串
'::'.join(pieces)

'a::b::guido'

In [126]:
# in关键字检测子字符串
'a' in pieces

True

In [127]:
# count方法返回特定子字符串出现的次数
val.count(',')

2

In [129]:
# replace方法 替代
val.replace(',', '::')

'a::b::  guido'

### 7.3.2 正则表达式

In [132]:
import re

text = "foo   bar\t baz   \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [135]:
# 上面的首先会被编译成一个模式，然后正则表达式的split方法在传入文本上被调用.
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [136]:
# 获得与模式匹配的列表
regex.findall(text)

['   ', '\t ', '   \t']

In [138]:
# findall 返回的是字符串中所有的匹配项
# search 返回的仅仅是第一个匹配项
# match 只在字符串的起始位置进行匹配
# sub 会将原字符串中的模式被新的字符串替代

text = """Dave dave@googlge.com
Steve steve@gmail.com
Rob rob@google.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE 使正则表达式不区分大小写
regex = re.compile(pattern, flags=re.IGNORECASE)

In [139]:
regex.findall(text)

['dave@googlge.com', 'steve@gmail.com', 'rob@google.com', 'ryan@yahoo.com']

In [143]:
# search 返回的匹配对象只告诉模式在字符串的起始和结束位置
m = regex.search(text) 
m

<re.Match object; span=(5, 21), match='dave@googlge.com'>

In [144]:
text[m.start(): m.end()]

'dave@googlge.com'

In [146]:
# match 只从字符串起始位置开始匹配，没有则返回NULL
print(regex.match(text))

None


In [147]:
regex.sub('SELECTED', text)

'Dave SELECTED\nSteve SELECTED\nRob SELECTED\nRyan SELECTED\n'

In [148]:
# 要查找三个部分，可以分别括起来
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [149]:
# groups方法 返回模式组件的元组
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [150]:
regex.findall(text)

[('dave', 'googlge', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'google', 'com'),
 ('ryan', 'yahoo', 'com')]

In [151]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: googlge, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: google, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 7.3.3 pandas中的向量化字符串函数

In [152]:
data = {'Dave': 'dave@googlge.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@google.com', 'Ryan': np.nan }
data = pd.Series(data)
data

Dave     dave@googlge.com
Steve     steve@gmail.com
Rob        rob@google.com
Ryan                  NaN
dtype: object

In [153]:
# data.map在NaN上会失效
# Series有面向数组的方法用于跳过NA值的字符串操作，这些方法通过Series的str属性进行调用

In [155]:
# contains
data.str.contains('gmail')

Dave     False
Steve     True
Rob      False
Ryan       NaN
dtype: object

In [156]:
# re
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, googlge, com)]
Steve     [(steve, gmail, com)]
Rob        [(rob, google, com)]
Ryan                        NaN
dtype: object

In [157]:
# str.get或str属性内部索引
data.str.get(1)

Dave       a
Steve      t
Rob        o
Ryan     NaN
dtype: object

In [158]:
data.str[1]

Dave       a
Steve      t
Rob        o
Ryan     NaN
dtype: object

In [159]:
# 切片
data.str[2:5]

Dave     ve@
Steve    eve
Rob      b@g
Ryan     NaN
dtype: object